# WallStreetBests: Reddit Sentiment $\implies$ Stock Alpha

In [25]:
import pandas as pd

# Adjust path in order to access preprocessing code
import sys
import os
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(parent_dir)
import src.preprocess as preprocess

## 1: Quick data audit

In [ ]:
# Read in Reddit data as pd.DataFrame
reddit = pd.read_csv('../data/reddit_wsb_parsed.csv')
reddit.head()

,title,score,id,url,comms_num,created,body,timestamp,ticker
0,"It's not about the money, it's about sending a...",55,l6ulcx,https://v.redd.it/6j75regs72e61,6,1.611863e+09,NaN,2021-01-28 21:37:41,None
1,Math Professor Scott Steiner says the numbers ...,110,l6uibd,https://v.redd.it/ah50lyny62e61,23,1.611862e+09,NaN,2021-01-28 21:32:10,None
2,Exit the system,0,l6uhhn,https://www.reddit.com/r/wallstreetbets/commen...,47,1.611862e+09,The CEO of NASDAQ pushed to halt trading “to g...,2021-01-28 21:30:35,None
3,NEW SEC FILING FOR GME! CAN SOMEONE LESS RETAR...,29,l6ugk6,https://sec.report/Document/0001193125-21-019848/,74,1.611862e+09,NaN,2021-01-28 21:28:57,None
4,"Not to distract from GME, just thought our AMC...",71,l6ufgy,https://i.redd.it/4h2sukb662e61.jpg,156,1.611862e+09,NaN,2021-01-28 21:26:56,None


### 1a: Null counts per column

In [27]:
# For each column
for col in reddit.columns:
    # Count returns the number of non null values in an object
    print(f'{col}: {reddit[col].count()}')

title: 53187
score: 53187
id: 53187
url: 53187
comms_num: 53187
created: 53187
body: 24738
timestamp: 53187
ticker: 0


### 1b: Top-10 tickers by post count and by up-vote score.

## 2: Daily sentiment feature

### 2a: Use VADER (or FinBERT for finance-tuned) to score each post

### 2b: Aggregate to sent_mean, sent_median, bull_ratio by ticker-date.